<a href="https://colab.research.google.com/github/MohanZhu0623/Sentiment_Analysis/blob/main/ML(KNN%2CRF%2CSVM%2CANN)_TF_IDF_ThreeClasses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import time

In [ ]:
!pip install gensim
from gensim.models import Word2Vec

ERROR: Operation cancelled by user


In [ ]:
!pip install xgboost
from xgboost import XGBClassifier

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_excel('/python_labelled_data.xlsx')

In [ ]:
# Preprocess data
def pre_process_data(dataset):
    # Convert to lowercase
    dataset['text'] = dataset['text'].str.lower()
    # Remove numbers
    dataset['text'] = dataset['text'].str.replace(r'\d+', '', regex=True)
    # Remove punctuation
    dataset['text'] = dataset['text'].str.replace(f"[{string.punctuation}]", "", regex=True)
    # Remove whitespace
    dataset['text'] = dataset['text'].str.strip()
    # Remove stopwords except 'not'
    stop_words = set(stopwords.words('english'))
    stop_words.remove('not')
    dataset['text'] = dataset['text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))
    return dataset

In [ ]:
# Apply preprocessing to the data
data = pre_process_data(data)
print(data.head())

                                                text  class
0  project designed help protect environment usin...      1
1  help us built sustainable studio eliminate cla...      1
2  paint something dont want explain isbob ross b...      0
3  free app allow pool reservations others get gr...      1
4  prohibition themed gastro pub dark silent head...      1


In [ ]:
# Create TF-IDF vectorizer for unigrams and bigrams
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3)
X_tfidf = tfidf_vectorizer.fit_transform(data['text'])

# Convert to DataFrame
corpus_clean = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
# Combine class label with features
labeled_dtm = pd.concat([data[['class']], corpus_clean], axis=1)

In [ ]:

labeled_dtm = labeled_dtm.loc[:, ~labeled_dtm.columns.duplicated()]

print(labeled_dtm.columns.duplicated().sum())
class_column = labeled_dtm['class'].squeeze()
print(class_column.shape)
print(class_column.value_counts())

0
(2269,)
class
 1    962
 0    955
-1    352
Name: count, dtype: int64


In [ ]:
# Split the labeled DTM into training set (80%) and hold-out test set (20%)
partition = 0.8
train_labeled, test_labeled = train_test_split(labeled_dtm, test_size=1-partition, random_state=128, stratify=labeled_dtm['class'])

In [ ]:
print(f"Training set size: {train_labeled.shape}")
print(f"Test set size: {test_labeled.shape}")

Training set size: (1815, 2328)
Test set size: (454, 2328)


In [ ]:
# Set cross-validation parameters
cv_tune = 5
rep_tune = 1
cv_final = 10
rep_final = 5

# Define cross-validation strategies
cv_strategy_tune = RepeatedKFold(n_splits=cv_tune, n_repeats=rep_tune, random_state=128)
cv_strategy_final = RepeatedKFold(n_splits=cv_final, n_repeats=rep_final, random_state=128)

In [ ]:
# Define parameter grids
param_grid_knn = {'n_neighbors': [1, 15, 30, 45, 65]}
param_grid_rf = {'n_estimators': [100], 'max_features': [round(np.sqrt(train_labeled.shape[1]) / 2), round(np.sqrt(train_labeled.shape[1]))], 'min_samples_split': [2]}
param_grid_svm = {'C': [0.01, 0.1, 1, 10, 100]}
param_grid_ann = {'hidden_layer_sizes': [(50, 50), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam']}


# Define models
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
svm = SVC()
ann = MLPClassifier(max_iter=1000)

# Combine models and parameter grids
models = {
    'knn': (knn, param_grid_knn),
    'rf': (rf, param_grid_rf),
    'svm': (svm, param_grid_svm),
    'ann': (ann, param_grid_ann),
}

# Initialize results dataframe
results = []


from sklearn.metrics import f1_score, precision_score, recall_score
# Define a function to evaluate the model on the test set
def evaluate_model_on_test(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    conf_matrix = confusion_matrix(y_test, predictions)
    return accuracy, f1, precision, recall, conf_matrix


In [ ]:
# Train and tune models
for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv_strategy_tune, verbose=2, n_jobs=-1)
    start_time = time.time()
    grid_search.fit(train_labeled.drop(columns=['class']), train_labeled['class'])
    end_time = time.time()

    # Save results
    best_model = grid_search.best_estimator_
    train_acc = grid_search.best_score_
    tuned_parameters = grid_search.best_params_
    runtime = end_time - start_time

    results.append({
        'final_model': best_model,
        'model': model_name,
        'train_acc': train_acc,
        'tuned_parameters': tuned_parameters,
        'runtime': runtime
    })

    # Fit tuned model on full dataset
    print(f"Fitting final {model_name} model on full dataset...")
    final_model = model
    final_model.fit(labeled_dtm.drop(columns=['class']), labeled_dtm['class'])
    repeated_acc = final_model.score(labeled_dtm.drop(columns=['class']), labeled_dtm['class'])
    results[-1]['repeated_acc'] = repeated_acc

    # Evaluate the model on the test set
    test_acc, f1, precision, recall, test_conf_matrix = evaluate_model_on_test(best_model, test_labeled.drop(columns=['class']), test_labeled['class'])
    results[-1]['test_acc'] = test_acc
    results[-1]['f1'] = f1
    results[-1]['precision'] = precision
    results[-1]['recall'] = recall
    results[-1]['test_conf_matrix'] = test_conf_matrix

Training knn...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting final knn model on full dataset...
Training rf...
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting final rf model on full dataset...
Training svm...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting final svm model on full dataset...
Training ann...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting final ann model on full dataset...


In [ ]:
# Convert results to DataFrame
df_train_results = pd.DataFrame(results, columns=["final_model", "model", "train_acc", "tuned_parameters", "runtime", "repeated_acc", "test_acc", "f1", "precision", "recall", "test_conf_matrix"])

# Display results
print(df_train_results)

                                         final_model model  train_acc  \
0               KNeighborsClassifier(n_neighbors=30)   knn   0.514050   
1  (DecisionTreeClassifier(max_features=24, rando...    rf   0.538292   
2                                          SVC(C=10)   svm   0.520661   
3  MLPClassifier(hidden_layer_sizes=(50, 50), max...   ann   0.519008   

                                    tuned_parameters      runtime  \
0                                {'n_neighbors': 30}     6.000789   
1  {'max_features': 24, 'min_samples_split': 2, '...    24.877025   
2                                          {'C': 10}   167.801672   
3  {'activation': 'relu', 'hidden_layer_sizes': (...  1436.745502   

   repeated_acc  test_acc        f1  precision    recall  \
0      0.687087  0.522026  0.515274   0.533110  0.522026   
1      0.988100  0.530837  0.526831   0.562224  0.530837   
2      0.972234  0.513216  0.509543   0.529292  0.513216   
3      0.988100  0.515419  0.515151   0.517881  